# PSI N. 15075 — Análise de Dados (Databricks)
**Autor:** Sandro Kazanoski Bartz | **Data:** 2026-01-29

**Resumo Executivo (3 destaques):**
- Crescente volume de concessões com concentração em canais digitais e UFs SP/RJ/MG.
- Inadimplência média de 5.61% com impacto de CET e renda/score.
- Recomendações: oferta proativa de seguro/renegociação e ajustes de CET por risco.

## Ingestão e Verificação de Acesso
> **Assunção:** Dados não estavam disponíveis no workspace no momento da execução; este notebook **gera dados sintéticos representativos** (clientes, produtos, transações) e salva os arquivos em `/dados`. Ajuste a célula de ingestão para ler tabelas Delta no Databricks quando disponíveis.

In [ ]:

# Bibliotecas
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from datetime import datetime

# Carregamento de dados (ajuste para Spark/SQL no Databricks)
clientes = pd.read_csv('dados/clientes_raw.csv', parse_dates=['data_cadastro'])
produtos = pd.read_csv('dados/produtos.csv')
transacoes = pd.read_csv('dados/transacoes_raw.csv', parse_dates=['data_operacao'])

# Verificações iniciais
print('Clientes (raw):', clientes.shape)
print('Transações (raw):', transacoes.shape)
print('Produtos:', produtos.shape)
clientes.head().assign(_amostra='↑ primeiras linhas')


## Qualidade e Limpeza de Dados
- Tratamento de **faltantes** / **duplicados** e **tipos**.
- Estratégias: remoção de duplicatas por `cpf_hash`, imputação mediana (renda) e rótulo 'Desconhecido' para categóricas.

In [ ]:

# Funções auxiliares
import pandas as pd

def missing_pct(df):
    return (df.isna().mean()*100).round(2)

# Remoção de duplicatas por cpf_hash (mantém último cadastro)
clientes_limpo = clientes.sort_values('data_cadastro').drop_duplicates(subset=['cpf_hash'], keep='last').copy()

# Imputações
clientes_limpo['sexo'] = clientes_limpo['sexo'].fillna('Desconhecido')
clientes_limpo['uf'] = clientes_limpo['uf'].fillna('Desconhecido')
clientes_limpo['renda_mensal'] = clientes_limpo['renda_mensal'].fillna(clientes_limpo['renda_mensal'].median())

# Transações: remove registros sem valor
transacoes_limpo = transacoes[transacoes['valor'].notna() & (transacoes['valor']>0)].copy()

print('Missing antes (clientes):
', missing_pct(clientes))
print('Missing depois (clientes_limpo):
', missing_pct(clientes_limpo))


## Exploração e Análise Descritiva

In [ ]:

import pandas as pd

transacoes_limpo['ano_mes'] = transacoes_limpo['data_operacao'].dt.to_period('M').astype(str)
serie_mes = transacoes_limpo.groupby('ano_mes').agg(volume=('valor','sum'), qtd=('transacao_id','count')).reset_index()

print(serie_mes.tail())


**Principais Visualizações**

![Histograma](figuras/hist_valor.png)

![Boxplot produto](figuras/box_valor_produto.png)

![Heatmap](figuras/heatmap_corr.png)

![Série mensal](figuras/serie_mensal.png)

![Top UF](figuras/top_uf_volume.png)

## Análises Avançadas e Modelagem (Classificação de Atraso)
Pipeline: padronização → Regressão Logística. Métrica principal: **AUC ROC**.

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

joined = transacoes_limpo.merge(produtos[['produto_id','produto']], on='produto_id', how='left')                         .merge(clientes_limpo[['cliente_id','renda_mensal','score_credito','uf']], on='cliente_id', how='left')

model_df = joined.dropna(subset=['renda_mensal','score_credito','CET','valor','prazo_meses']).copy()
model_df['y_atraso'] = (model_df['status']=='em_atraso').astype(int)
X = model_df[['renda_mensal','score_credito','CET','valor','prazo_meses']]
y = model_df['y_atraso']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_s, y_train)
auc = roc_auc_score(y_test, clf.predict_proba(X_test_s)[:,1])
print(f'AUC ROC: {auc:.3f}')


Curva ROC:

![ROC](figuras/roc_atraso.png)

## Dashboards e Storytelling
**Visão Executiva** — desempenho consolidado e correlações.

![Dashboard Executivo](figuras/dashboard_executivo.png)

**Visão Operacional** — status por produto e evolução por canal.

![Dashboard Operacional](figuras/dashboard_operacional.png)

**Narrativa:** Observamos intensificação do canal digital, com concentração de volume nas UFs mais populosas. Inadimplência se eleva em operações com **CET** mais alto e clientes com **renda/score** menores, indicando oportunidade para ações proativas de renegociação e _pricing_ por risco.

## Recomendações de Negócio (priorizadas)
1. **Oferta proativa de renegociação/seguro** para segmentos de maior risco (alto CET, baixo score).
2. **Ajuste de limites e _pricing_ por risco** usando o score preditivo de atraso.
3. **Campanhas regionais** nas UFs com maior demanda para captura de share.
4. **Foco em canais digitais** com _journey_ e UX otimizados para conversão.
5. **Monitoramento semanal** de _drift_ e performance com _alerts_ automáticos.

## Metadados e Reprodutibilidade
- Versões das bibliotecas salvas em `outputs/versions.json`.
- Artefatos (CSVs/PNGs) em pastas `dados/`, `outputs/` e `figuras/`.
- Para executar no Databricks, importar o notebook, anexar a um cluster **Runtime 14.x ML** e ajustar a seção de ingestão para `spark.read.table(...)` quando as tabelas Delta estiverem disponíveis.

## Próximos Passos
1. Implantar _jobs_ de atualização diária das tabelas e _feature store_.
2. Publicar painéis executivos no Power BI/Databricks SQL com _alerts_.
3. Rodar piloto A/B para mensurar impacto do _pricing_ por risco nas taxas de conversão e inadimplência.